In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import openpyxl
from datetime import datetime

In [35]:
# Configure Chrome DevTools options
chrome_options = Options()
chrome_options.add_experimental_option("w3c", False)
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--headless")

In [36]:
driver = webdriver.Chrome()

In [37]:
def collect_data(city, latitude, longitude):
    # Set Serach Location
    driver.execute_cdp_cmd('Emulation.setGeolocationOverride', {
        'latitude': latitude,
        'longitude': longitude,
        'accuracy': 100
    })
    
    # Navigate to search results page
    search_term = f"internet packages near {city}"
    driver.get(f"https://www.google.com/search?q={search_term}")
    
    # Wait for the page to load
    driver.implicitly_wait(1)
    
    # Collect data for sponsored links
    sponsored_links = driver.find_elements(By.XPATH, "//div[@class='uEierd']")

    data = []
    for link in sponsored_links:
        link_element = link.find_element(By.TAG_NAME, "a")
        link_text = link_element.text
        link_url = link_element.get_attribute("href")
        link_description = link.find_element(By.TAG_NAME, "span").text
        
        additional_data_elements = link.find_elements(By.CSS_SELECTOR, ".MUxGbd, .yDYNvb, .lyLwlc")
        additional_data = [elem.text for elem in additional_data_elements]
        
        data.append({
            "link_text": link_text,
            "link_url": link_url,
            "link_description": link_description,
            "additional_data": additional_data
        })
        
    return data

In [38]:
cities_excel_file = 'data.xlsx'
cities_df = pd.read_excel('cities_data.xlsx')

In [39]:
# Iterate over the cities and collect data for each city
results_data = []
for index, row in cities_df.iterrows():
    city = row["city"]
    latitude = row["latitude"]
    longitude = row["longitude"]

    data = collect_data(city, latitude, longitude)
    results_data.extend(data)

# Create a dataframe from the collected results data
results_df = pd.DataFrame(results_data)

# Add a timestamp column to the dataframe
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
results_df["timestamp"] = timestamp

# Append the results dataframe to the Excel sheet
results_sheet_name = "results"

try:
    with pd.ExcelWriter(cities_excel_file, mode="a", engine="openpyxl") as writer:
        results_df.to_excel(writer, sheet_name=results_sheet_name, index=False, header=not writer.book)

except FileNotFoundError:
    results_df.to_excel(cities_excel_file, sheet_name=results_sheet_name, index=False)

print("Data appended to Excel successfully.")

# Close the browser
driver.quit()

Data appended to Excel successfully.
